# Init

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datetime
import argparse
import os
import time

import torch
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms

import flow_transforms
from torch.utils.tensorboard import SummaryWriter

#  Импорт файлов датасета

In [ ]:
import zipfile

with zipfile.ZipFile(file='./drive/MyDrive/test_chairs.zip') as zipnik:
  zipnik.extractall()

# Константы

In [ ]:
# value by which flow will be divided. Original value is 20 but 1 with
# batchNorm gives good results
DIV_FLOW = 20

best_EPE = -1

start_epoch = 0
n_iter = start_epoch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_RATIO = 0.8
BATCH_SIZE = 8
NUM_WORKERS = 2
EPOCH_SIZE = 1000
EPOCHS = 30
LR = 10e-4
MOMENTUM = 0.9
BETA = 0.999
WEIGHT_DECAY = 4e-4
BIAS_DECAY = 0
MULTISCALE_WEIGHTS = [0.005,0.01,0.02,0.08,0.32]
PRINT_FREQ = 10
MILESTONES = [10, 20]


save_path = 'model_'

# Запись прогресса обучения ?

In [ ]:
train_writer = SummaryWriter(os.path.join(save_path,'train'))
test_writer = SummaryWriter(os.path.join(save_path,'test'))
output_writers = []
for i in range(3):
  output_writers.append(SummaryWriter(os.path.join(save_path,'test',str(i))))


# Преобразования для загрузки изображений датасета

In [ ]:
 # Data loading code
input_transform = transforms.Compose([
        flow_transforms.ArrayToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
        transforms.Normalize(mean=[0.45,0.432,0.411], std=[1,1,1])
    ])
target_transform = transforms.Compose([
        flow_transforms.ArrayToTensor(),
        transforms.Normalize(mean=[0,0],std=[DIV_FLOW, DIV_FLOW])
    ])

# Создаём пайплайн [преобразований изображений](https://github.com/DimaKurd/FlowNetPytorch/blob/master/flow_transforms.py) для обучения

In [ ]:
co_transform = flow_transforms.Compose([
            flow_transforms.RandomTranslate(10),
            flow_transforms.RandomRotate(10,5),
            flow_transforms.RandomCrop((320,448)),
            flow_transforms.RandomVerticalFlip(),
            flow_transforms.RandomHorizontalFlip()
        ])


# Грузим данные, создаём даталоадеры

In [12]:
import os.path
import glob
from listdataset import ListDataset
from util_dataset import split2list
import numpy as np



def make_dataset(dir, split=None):
    '''Will search for triplets that go by the pattern '[name]_img1.ppm  [name]_img2.ppm    [name]_flow.flo' '''
    images = []
    for flow_map in sorted(glob.glob(os.path.join(dir,'*_flow.flo'))):
        flow_map = os.path.basename(flow_map)
        root_filename = flow_map[:-9]
        img1 = root_filename+'_img1.ppm'
        img2 = root_filename+'_img2.ppm'
        if not (os.path.isfile(os.path.join(dir,img1)) and os.path.isfile(os.path.join(dir,img2))):
            continue

        images.append([[img1,img2],flow_map])

    return split2list(images, split)


def flying_chairs(root, transform=None, target_transform=None,
                  co_transform=None, split=None):
    train_list, test_list = make_dataset(root, split)
    train_dataset = ListDataset(root, train_list, transform, target_transform, co_transform)
    test_dataset = ListDataset(root, test_list, transform, target_transform)

    return test_dataset

In [13]:
test_set = flying_chairs(
        'test',
        transform=input_transform,
        target_transform=target_transform,
        co_transform=co_transform,
        split = 0
    )

In [14]:
len(test_set)

165

In [15]:
val_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        shuffle=False)


# Объявление модели

In [23]:
import torch
import torch.nn as nn
from torch.nn.init import kaiming_normal_, constant_
from util_model import conv, predict_flow, deconv, crop_like

class FlowNetS(nn.Module):
    expansion = 1

    def __init__(self,batchNorm=True):
        super(FlowNetS,self).__init__()

        self.batchNorm = batchNorm
        self.conv1   = conv(self.batchNorm,   6,   64, kernel_size=7, stride=2)
        self.conv2   = conv(self.batchNorm,  64,  128, kernel_size=5, stride=2)
        self.conv3   = conv(self.batchNorm, 128,  256, kernel_size=5, stride=2)
        self.conv3_1 = conv(self.batchNorm, 256,  256)
        self.conv4   = conv(self.batchNorm, 256,  512, stride=2)
        self.conv4_1 = conv(self.batchNorm, 512,  512)
        self.conv5   = conv(self.batchNorm, 512,  512, stride=2)
        self.conv5_1 = conv(self.batchNorm, 512,  512)
        self.conv6   = conv(self.batchNorm, 512, 1024, stride=2)
        self.conv6_1 = conv(self.batchNorm,1024, 1024)

        self.deconv5 = deconv(1024,512)
        self.deconv4 = deconv(1026,256)
        self.deconv3 = deconv(770,128)
        self.deconv2 = deconv(386,64)

        self.predict_flow6 = predict_flow(1024)
        self.predict_flow5 = predict_flow(1026)
        self.predict_flow4 = predict_flow(770)
        self.predict_flow3 = predict_flow(386)
        self.predict_flow2 = predict_flow(194)

        self.upsampled_flow6_to_5 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow5_to_4 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow4_to_3 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)
        self.upsampled_flow3_to_2 = nn.ConvTranspose2d(2, 2, 4, 2, 1, bias=False)

        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                kaiming_normal_(m.weight, 0.1)
                if m.bias is not None:
                    constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                constant_(m.weight, 1)
                constant_(m.bias, 0)

    def forward(self, x):
        out_conv2 = self.conv2(self.conv1(x))
        out_conv3 = self.conv3_1(self.conv3(out_conv2))
        out_conv4 = self.conv4_1(self.conv4(out_conv3))
        out_conv5 = self.conv5_1(self.conv5(out_conv4))
        out_conv6 = self.conv6_1(self.conv6(out_conv5))

        flow6       = self.predict_flow6(out_conv6)
        flow6_up    = crop_like(self.upsampled_flow6_to_5(flow6), out_conv5)
        out_deconv5 = crop_like(self.deconv5(out_conv6), out_conv5)

        concat5 = torch.cat((out_conv5,out_deconv5,flow6_up),1)
        flow5       = self.predict_flow5(concat5)
        flow5_up    = crop_like(self.upsampled_flow5_to_4(flow5), out_conv4)
        out_deconv4 = crop_like(self.deconv4(concat5), out_conv4)

        concat4 = torch.cat((out_conv4,out_deconv4,flow5_up),1)
        flow4       = self.predict_flow4(concat4)
        flow4_up    = crop_like(self.upsampled_flow4_to_3(flow4), out_conv3)
        out_deconv3 = crop_like(self.deconv3(concat4), out_conv3)

        concat3 = torch.cat((out_conv3,out_deconv3,flow4_up),1)
        flow3       = self.predict_flow3(concat3)
        flow3_up    = crop_like(self.upsampled_flow3_to_2(flow3), out_conv2)
        out_deconv2 = crop_like(self.deconv2(concat3), out_conv2)

        concat2 = torch.cat((out_conv2,out_deconv2,flow3_up),1)
        flow2 = self.predict_flow2(concat2)

        if self.training:
            return flow2,flow3,flow4,flow5,flow6
        else:
            return flow2

    def weight_parameters(self):
        return [param for name, param in self.named_parameters() if 'weight' in name]

    def bias_parameters(self):
        return [param for name, param in self.named_parameters() if 'bias' in name]


/content/util_model.py:10: ImportWarning: failed to load custom correlation modulewhich is needed for FlowNetC
  warnings.warn("failed to load custom correlation module"


# Инициализируем модель

In [25]:
model = FlowNetS(batchNorm=False).to(device)
checkpoint = torch.load('drive/MyDrive/flownets.pth.tar')

In [30]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [29]:
checkpoint['state_dict']

OrderedDict([('conv1.0.weight',
              tensor([[[[-1.1351e-01,  3.4869e-03,  8.2545e-02,  ...,  1.9093e-02,
                         -8.9840e-02,  3.2131e-02],
                        [ 1.4271e-01, -5.7017e-02, -3.7536e-02,  ..., -7.8777e-02,
                         -1.3997e-01,  3.3790e-02],
                        [-8.1992e-02,  1.2495e-02,  1.7249e-02,  ..., -1.2448e-01,
                         -2.5524e-02, -1.4824e-01],
                        ...,
                        [ 3.8428e-02,  1.1729e-01,  4.3418e-02,  ..., -3.5150e-02,
                          1.0059e-01,  6.0759e-02],
                        [-3.3551e-02, -7.2201e-02,  4.9068e-02,  ..., -3.3621e-03,
                         -2.8151e-02, -1.1709e-01],
                        [ 5.5817e-03,  3.7195e-02, -3.8061e-02,  ...,  6.6941e-02,
                         -6.8176e-02, -1.4509e-01]],
              
                       [[ 2.1285e-01, -1.2783e-01, -6.6672e-02,  ..., -1.2325e-02,
                          1.16

# Validate code

In [37]:
from util import flow2rgb, AverageMeter, save_checkpoint
from multiscaleloss import multiscaleEPE, realEPE

def validate(val_loader, model, epoch, output_writers):


    batch_time = AverageMeter()
    flow2_EPEs = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.to(device)
        input = torch.cat(input,1).to(device)

        # compute output
        output = model(input)
        flow2_EPE = DIV_FLOW*realEPE(output, target, sparse=False)
        # record EPE
        flow2_EPEs.update(flow2_EPE.item(), target.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i < len(output_writers):  # log first output of first batches
            if epoch == start_epoch:
                mean_values = torch.tensor([0.45,0.432,0.411], dtype=input.dtype).view(3,1,1)
                output_writers[i].add_image('GroundTruth', flow2rgb(DIV_FLOW * target[0], max_value=10), 0)
                output_writers[i].add_image('Inputs', (input[0,:3].cpu() + mean_values).clamp(0,1), 0)
                output_writers[i].add_image('Inputs', (input[0,3:].cpu() + mean_values).clamp(0,1), 1)
            output_writers[i].add_image('FlowNet Outputs', flow2rgb(DIV_FLOW * output[0], max_value=10), epoch)

        if i % PRINT_FREQ == 0:
            print('Test: [{0}/{1}]\t Time {2}\t EPE {3}'
                  .format(i, len(val_loader), batch_time, flow2_EPEs))

    print(' * EPE {:.3f}'.format(flow2_EPEs.avg))

    return flow2_EPEs.avg


In [38]:
best_EPE = validate(val_loader, model, 0, output_writers)

Test: [0/21]	 Time 1.004 (1.004)	 EPE 12.109 (12.109)
Test: [10/21]	 Time 0.149 (0.373)	 EPE 14.756 (11.007)
Test: [20/21]	 Time 0.072 (0.352)	 EPE 6.043 (10.555)
 * EPE 10.555
